## Installs and imports

In [1]:
!pip install --upgrade pip
!pip install sentencepiece
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import numpy as np
from scipy.special import softmax

## Data

In [3]:
def preprocess(corpus):
  outcorpus = []
  for text in corpus:
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    new_text = " ".join(new_text)
    outcorpus.append(new_text)
  return outcorpus

In [4]:
!wget https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/all/test_text.txt

--2025-04-14 22:29:33--  https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/all/test_text.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 654172 (639K) [text/plain]
Saving to: ‘test_text.txt’

test_text.txt       100%[===================>] 638.84K  --.-KB/s    in 0.05s   

2025-04-14 22:29:34 (11.6 MB/s) - ‘test_text.txt’ saved [654172/654172]



In [5]:
dataset_path = './test_text.txt'
dataset = open(dataset_path).read().split('\n')

In [6]:
# this is a dataset in 8 different languages
for example in [0,870,1740,2610,3480,4350,5220,6090]:
  print(dataset[example])

نوال الزغبي (الشاب خالد ليس عالمي) هههههههه أتفرجي على ها الفيديو يا مبتدئة http vía @user
Trying to have a conversation with my dad about vegetarianism is the most pointless infuriating thing ever #caveman 
Royal: le président n'aime pas les pauvres? "c'est n'importe quoi" http …
@user korrekt! Verstehe sowas nicht...
CONGRESS na ye party kabhi bani hoti na india ka partition hota nd na hi humari country itni khokhli hoti   @ 
@user @user Ma Ferrero? il compagno Ferrero? ma il suo partito esiste ancora? allora stiamo proprio frecati !!!
todos os meus favoritos na prova de eliminação #MasterChefBR
@user jajajaja dale, hacete la boluda vos jajaja igual a vos nunca se te puede tomar en serio te mando un abrazo desde Perú!


## Model

In [7]:
CUDA = True # set to true if using GPU (Runtime -> Change runtime Type -> GPU)
BATCH_SIZE = 32
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
config = AutoConfig.from_pretrained(MODEL) # used for id to label name
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
if CUDA:
  model = model.to('cuda')
_ = model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

## Forward

In [8]:
def forward(text, cuda=True):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
  if cuda:
    encoded_input.to('cuda')
    output = model(**encoded_input)
    scores = output[0].detach().cpu().numpy()
  else:
    output = model(**encoded_input)
    scores = output[0].detach().numpy()

  scores = softmax(scores, axis=-1)
  return scores

In [9]:
dl = DataLoader(dataset, batch_size=BATCH_SIZE)
all_preds = []
for idx,batch in enumerate(dl):
  print('Batch ',idx+1,' of ',len(dl))
  text = preprocess(batch)
  scores = forward(text, cuda=CUDA)
  preds = np.argmax(scores, axis=-1)
  all_preds.extend(preds)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batch  1  of  218
Batch  2  of  218
Batch  3  of  218
Batch  4  of  218
Batch  5  of  218
Batch  6  of  218
Batch  7  of  218
Batch  8  of  218
Batch  9  of  218
Batch  10  of  218
Batch  11  of  218
Batch  12  of  218
Batch  13  of  218
Batch  14  of  218
Batch  15  of  218
Batch  16  of  218
Batch  17  of  218
Batch  18  of  218
Batch  19  of  218
Batch  20  of  218
Batch  21  of  218
Batch  22  of  218
Batch  23  of  218
Batch  24  of  218
Batch  25  of  218
Batch  26  of  218
Batch  27  of  218
Batch  28  of  218
Batch  29  of  218
Batch  30  of  218
Batch  31  of  218
Batch  32  of  218
Batch  33  of  218
Batch  34  of  218
Batch  35  of  218
Batch  36  of  218
Batch  37  of  218
Batch  38  of  218
Batch  39  of  218
Batch  40  of  218
Batch  41  of  218
Batch  42  of  218
Batch  43  of  218
Batch  44  of  218
Batch  45  of  218
Batch  46  of  218
Batch  47  of  218
Batch  48  of  218
Batch  49  of  218
Batch  50  of  218
Batch  51  of  218
Batch  52  of  218
Batch  53  of  218
Ba

In [10]:
# this is a dataset in 8 different languages
for example in [0,870,1740,2610,3480,4350,5220,6090]:
  pred = all_preds[example]
  print(dataset[example], '--->', config.id2label[pred])

نوال الزغبي (الشاب خالد ليس عالمي) هههههههه أتفرجي على ها الفيديو يا مبتدئة http vía @user ---> neutral
Trying to have a conversation with my dad about vegetarianism is the most pointless infuriating thing ever #caveman  ---> negative
Royal: le président n'aime pas les pauvres? "c'est n'importe quoi" http … ---> negative
@user korrekt! Verstehe sowas nicht... ---> negative
CONGRESS na ye party kabhi bani hoti na india ka partition hota nd na hi humari country itni khokhli hoti   @  ---> negative
@user @user Ma Ferrero? il compagno Ferrero? ma il suo partito esiste ancora? allora stiamo proprio frecati !!! ---> negative
todos os meus favoritos na prova de eliminação #MasterChefBR ---> positive
@user jajajaja dale, hacete la boluda vos jajaja igual a vos nunca se te puede tomar en serio te mando un abrazo desde Perú! ---> neutral
